# Simple DPS calculator for Pathfinder CRPG

How to use
1. Use Run -> Run All Cells
2. Scroll all the way down, input your numbers.

Use "Comparison mode" to compare two damage setups.

In [61]:
def calc_chances(required_ac, crit_range_threshold = 20, crit_autoconfirm = False, crit_confirm_bonus = 0):
    effective_ac = required_ac if required_ac < 20 else 20
    effective_ac = 2 if effective_ac < 2 else effective_ac
    effective_crit_ac = required_ac - crit_confirm_bonus if required_ac - crit_confirm_bonus < 20 else 20
    effective_crit_ac = 2 if effective_crit_ac < 2 else effective_crit_ac
    crit_range = crit_range_threshold if crit_range_threshold >= effective_ac else effective_ac
    total = 0    
    crit_confirm_chance = (21.0 - effective_crit_ac) / 20
    if crit_autoconfirm:
        crit_confirm_chance = 1
    return ((21.0 - effective_ac ) / 20, (21.0 - crit_range) / 20, crit_confirm_chance)

def calc_damage(required_ac, damage, crit_range_threshold = 20, crit_mult = 2, crit_autoconfirm = False, crit_confirm_bonus = 0):
    to_hit, crit_chance , crit_confirm_chance = calc_chances(required_ac, crit_range_threshold, crit_autoconfirm, crit_confirm_bonus)
    total = to_hit + crit_chance * crit_confirm_chance * (crit_mult - 1)
    return damage * total

def calculate_series(attacks, enemy_ac, damage, crit_range_threshold, crit_mult, crit_autoconfirm, crit_confirm_bonus):
    total = 0
    for atk in attacks:
        total += calc_damage(enemy_ac - atk, damage, crit_range_threshold, crit_mult, crit_autoconfirm, crit_confirm_bonus)
    return total

In [62]:
import matplotlib.pyplot as plt
import numpy as np

try:
    import piplite
    await piplite.install(['ipywidgets'])
except ImportError:
    pass

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [63]:
class Attack:
     def __init__(self, ab: int = 0, dmg: float = 1, add_non_crit_dmg: float = 0, crit_range: int = 20, crit_mult: int = 2, crit_autoconfirm: bool = False, crit_confirm_bonus: int = 0):
        self.ab = ab
        self.dmg = dmg
        self.add_non_crit_dmg = add_non_crit_dmg
        self.crit_range = crit_range
        self.crit_mult = crit_mult
        self.crit_autoconfirm = crit_autoconfirm
        self.crit_confirm_bonus = crit_confirm_bonus
        
class FullAttack:
    def __init__(self, ab: int, add_attacks: list[Attack], iteratives: int, bonus_attacks: int, twf_attacks: int
                 , twf_penalty: int, dmg: float, add_non_crit_dmg: float, crit_range: int, crit_mult: int, crit_autoconfirm: bool, crit_confirm_bonus: int, label):
        self.ab = ab
        self.add_attacks = add_attacks
        self.iteratives = iteratives
        self.bonus_attacks = bonus_attacks
        self.twf_attacks = twf_attacks
        self.dmg = dmg
        self.add_non_crit_dmg = add_non_crit_dmg
        self.crit_range = crit_range
        self.crit_mult = crit_mult
        self.crit_autoconfirm = crit_autoconfirm
        self.crit_confirm_bonus = crit_confirm_bonus
        self.label = label

In [64]:
def calc_damage_atk(enemy_ac: int, attack: Attack):
    to_hit, crit_chance , crit_confirm_chance = calc_chances(enemy_ac - attack.ab, attack.crit_range, attack.crit_autoconfirm, attack.crit_confirm_bonus)
    total = to_hit + crit_chance * crit_confirm_chance * (attack.crit_mult - 1)
    return attack.dmg * total + to_hit * attack.add_non_crit_dmg

def calc_damage_fatk(enemy_ac: int, attack: FullAttack):
    to_hit, crit_chance , crit_confirm_chance = calc_chances(enemy_ac, attack.crit_range, attack.crit_autoconfirm, attack.crit_confirm_bonus)
    total = to_hit + crit_chance * crit_confirm_chance * (attack.crit_mult - 1)
    return attack.dmg * total + to_hit * attack.add_non_crit_dmg

def calculate_full_attack(attacks, enemy_ac: int, full_attack: FullAttack):
    total = 0
    for atk in attacks:
        total += calc_damage_fatk(enemy_ac - atk, full_attack)
    for batk in full_attack.add_attacks:
        total += calc_damage_atk(enemy_ac, batk)
    return total

def get_attacks(full_attack: FullAttack):
    attack_list : list[int] = [full_attack.ab]
    if full_attack.iteratives > 0:
        for x in range(1, full_attack.iteratives + 1):
            attack_list.append(full_attack.ab - x * 5)
    if full_attack.bonus_attacks > 0:
        for x in range(1, full_attack.bonus_attacks + 1):
            attack_list.append(full_attack.ab)
    if full_attack.twf_attacks > 0:
        for x in range(0, full_attack.twf_attacks):
            attack_list.append(full_attack.ab - full_attack.twf_penalty- x * 5)
    return np.array(attack_list)

In [65]:
def calculate_dps_obj(full_attack_1: FullAttack, draw_2: bool, full_attack_2: FullAttack):    
    attacks = get_attacks(full_attack_1)
    ac_min = attacks.min()
    ac_max = (attacks.max()) + 21
    

    if draw_2:
        attacks_2 = get_attacks(full_attack_2)
        ac_min = min(attacks_2.min(), ac_min)
        ac_max = max(attacks_2.max() + 21, ac_max)
        
    enemy_ac_range = np.arange(ac_min, ac_max, 1)
    
    dmg_distr = np.array([calculate_full_attack(attacks, xi, full_attack_1) for xi in enemy_ac_range])            
    plt.plot(enemy_ac_range, dmg_distr, label=full_attack_1.label)
    if draw_2:
        dmg_distr_2 = np.array([calculate_full_attack(attacks_2, xi, full_attack_2) for xi in enemy_ac_range])            
        plt.plot(enemy_ac_range, dmg_distr_2, label=full_attack_2.label)
    
    plt.xlabel('enemy AC')
    plt.ylabel('damage')
    plt.grid(True)
    plt.legend()
    plt.title('Damage per full attack')
    plt.ylim(ymin=0)
    plt.show()    
        
    if draw_2:
        coef_dmg = dmg_distr_2 / dmg_distr
        plt.plot(enemy_ac_range, coef_dmg, label=full_attack_2.label + "/" + full_attack_1.label)
        plt.xlabel('enemy AC')        
        plt.ylabel('coef')
        plt.axhline(y = 1.0, color = 'r', linestyle = '-')
        plt.grid(True)
        plt.legend()
        plt.title('Dps difference')
        plt.ylim(ymin=0)
        plt.show()    
          
     
    

In [66]:
ab = widgets.BoundedIntText(value=0, min=-100, max=400, description='Total AB:')
iteratives = widgets.BoundedIntText(value=0, min=0, max=10, description='Iteratives:')
bonus_attacks = widgets.BoundedIntText(value=0, min=0, max=20, description='Bonus attacks:')
twf_attacks = widgets.BoundedIntText(value=0, min=0, max=3, description='TWF attacks:')
twf_penalty = widgets.BoundedIntText(value=0, min=0, max=12, description='TWF Penalty:')
crit_range = widgets.BoundedIntText(value=20, min=0, max=20, description='crit range:')
crit_mult = widgets.BoundedIntText(value=2, min=1, max=10, description='crit mult:')
crit_autoconfirm = widgets.Checkbox(value=False, description='crit autoconfirm')
crit_confirm_bonus = widgets.BoundedIntText(value=0, min=-100, max=100, description='crit confirm bonus:')

dmg = widgets.BoundedFloatText(value=1, min=1, max=1000, description='Damage:')
add_non_crit_dmg = widgets.BoundedFloatText(value=0, min=0, max=1000, description='Non-crit:')

graph_label=widgets.Text(value='setup1', placeholder='Graph label')

atk_ui = widgets.Box([ab, iteratives, bonus_attacks, twf_attacks, twf_penalty])
crit_ui = widgets.Box([crit_range, crit_mult, crit_autoconfirm, crit_confirm_bonus])
dmg_ui = widgets.Box([dmg, add_non_crit_dmg, graph_label])

ui = widgets.VBox([atk_ui, crit_ui, dmg_ui])


In [67]:
ab_c = widgets.BoundedIntText(value=0, min=-100, max=400, description='Total AB:')
iteratives_c = widgets.BoundedIntText(value=0, min=0, max=10, description='Iteratives:')
bonus_attacks_c = widgets.BoundedIntText(value=0, min=0, max=20, description='Bonus attacks:')
twf_attacks_c = widgets.BoundedIntText(value=0, min=0, max=3, description='TWF attacks:')
twf_penalty_c = widgets.BoundedIntText(value=0, min=0, max=12, description='TWF Penalty:')
crit_range_c = widgets.BoundedIntText(value=20, min=0, max=20, description='crit range:')
crit_mult_c = widgets.BoundedIntText(value=2, min=1, max=10, description='crit mult:')
crit_autoconfirm_c = widgets.Checkbox(value=False, description='crit autoconfirm')
crit_confirm_bonus_c = widgets.BoundedIntText(value=0, min=-100, max=100, description='crit confirm bonus:')

dmg_c = widgets.BoundedFloatText(value=1, min=1, max=1000, description='Damage:')
add_non_crit_dmg_c = widgets.BoundedFloatText(value=0, min=0, max=1000, description='Non-crit:')

graph_label_c = widgets.Text(value='setup2', placeholder='Graph label')

atk_ui_c = widgets.Box([ab_c, iteratives_c, bonus_attacks_c, twf_attacks_c, twf_penalty_c])
crit_ui_c = widgets.Box([crit_range_c, crit_mult_c, crit_autoconfirm_c, crit_confirm_bonus_c])
dmg_ui_c = widgets.Box([dmg_c, add_non_crit_dmg_c, graph_label_c])

ui_c = widgets.VBox([atk_ui_c, crit_ui_c, dmg_ui_c])


## Section below for adding custom attacks that don't fall under typical main hand

First block for setup 1, second block for setup 2

You will have to re-run cells for those changes to be applied, unlike widget changes, which are reflected immediately on graph

In [68]:
add_attack_1 = [
    # Attack(
    #     ab = 0,
    #     dmg = 10,
    #     add_non_crit_dmg = 0,
    #     crit_range = 20,
    #     crit_mult = 2,
    #     crit_autoconfirm = False,
    #     crit_confirm_bonus = 0
    # ),
    # Attack(
    #     ab = -5,
    #     dmg = 15,
    #     add_non_crit_dmg = 0,
    #     crit_range = 20,
    #     crit_mult = 2,
    #     crit_autoconfirm = False,
    #     crit_confirm_bonus = 0
    # )
]

In [69]:
add_attack_2 = []

In [70]:
comparison_mode = widgets.Checkbox(
    value=False,
    description='Comparison mode',
    disabled=False,
    indent=False
)

ui_total = widgets.VBox([ui, comparison_mode, ui_c])

def interact_fun(comparison_mode,
                 ab, iteratives, bonus_attacks, twf_attacks, twf_penalty, dmg, add_non_crit_dmg, crit_range, crit_mult, crit_autoconfirm, crit_confirm_bonus, graph_label,
                 ab_c, iteratives_c, bonus_attacks_c, twf_attacks_c, twf_penalty_c, dmg_c, add_non_crit_dmg_c, crit_range_c, crit_mult_c, crit_autoconfirm_c, crit_confirm_bonus_c, graph_label_c
                ):
    if comparison_mode:
        ui_c.layout.display = ''        
    else:
        ui_c.layout.display = 'none'
    
    atk1 = FullAttack(
        ab=ab,
        add_attacks=add_attack_1,
        iteratives=iteratives,
        bonus_attacks=bonus_attacks,
        twf_attacks=twf_attacks,
        twf_penalty=twf_penalty,
        dmg=dmg,
        add_non_crit_dmg=add_non_crit_dmg,
        crit_range=crit_range,
        crit_mult=crit_mult,
        crit_autoconfirm=crit_autoconfirm,
        crit_confirm_bonus=crit_confirm_bonus,
        label=graph_label
    )
    atk2 = FullAttack(
        ab=ab_c,
        add_attacks=[],
        iteratives=iteratives_c,
        bonus_attacks=bonus_attacks_c,
        twf_attacks=twf_attacks_c,
        twf_penalty=twf_penalty_c,
        dmg=dmg_c,
        add_non_crit_dmg=add_non_crit_dmg_c,
        crit_range=crit_range_c,
        crit_mult=crit_mult_c,
        crit_autoconfirm=crit_autoconfirm_c,
        crit_confirm_bonus=crit_confirm_bonus_c,
        label=graph_label_c
    )
    calculate_dps_obj(atk1, comparison_mode, atk2)
        

out = widgets.interactive_output(interact_fun,
                                 {
                                     'comparison_mode': comparison_mode,
                                     'ab': ab,
                                     'iteratives': iteratives,
                                     'bonus_attacks': bonus_attacks,
                                     'twf_attacks': twf_attacks,
                                     'twf_penalty': twf_penalty,
                                     'dmg': dmg,
                                     'add_non_crit_dmg': add_non_crit_dmg,
                                     'crit_range': crit_range,
                                     'crit_mult': crit_mult,
                                     'crit_autoconfirm': crit_autoconfirm,
                                     'crit_confirm_bonus': crit_confirm_bonus,
                                     'graph_label': graph_label,
                                     
                                     'ab_c': ab_c,
                                     'iteratives_c': iteratives_c,
                                     'bonus_attacks_c': bonus_attacks_c,
                                     'twf_attacks_c': twf_attacks_c,
                                     'twf_penalty_c': twf_penalty_c,
                                     'dmg_c': dmg_c,
                                     'add_non_crit_dmg_c': add_non_crit_dmg_c,
                                     'crit_range_c': crit_range_c,
                                     'crit_mult_c': crit_mult_c,
                                     'crit_autoconfirm_c': crit_autoconfirm_c,
                                     'crit_confirm_bonus_c': crit_confirm_bonus_c,
                                     'graph_label_c': graph_label_c
                                 })

## The actual calculator UI is below

In [71]:
display(ui_total, out)

Output()